In [ ]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time

In [8]:
def scrape(path):
    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(ChromeDriverManager().install())
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    driver.maximize_window()
    time.sleep(1)

    try:
        log = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//a[@class="js-login-form-open"])[1]')))
        log.click()
        
        provided_username = "fourbrotherstrading@icloud.com"
        username_field = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, "loginForm.Email")))
        username_field.send_keys(provided_username)
        
        provided_password = "Muhssan7865"
        pass_field = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, "loginForm.Password"))) #driver.find_element(By.ID, "loginForm.Password")
        pass_field.send_keys(provided_password)

        login_button = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, "//input[@value = 'Login']"))) #driver.find_element(By.XPATH, "//input[@value = 'Login']")
        login_button.click()
    except Exception as e:
        print(f"No login button found and error is --> {e}")
    
    try:
        sale_date = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, './/h4[text()="Sale date"]')))
        sale_date.click()
        
        tom = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, './/a[text()="Tomorrow"]')))
        tom.click()
    except Exception as e:
        print(f"Cookie banner handling error: {e}")
        
    time.sleep(2)
    result = []
    try:
        events =WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="calendar__item "]')))
        for event in events:
            auc_name = WebDriverWait(event, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="calendar_event_title"]'))) 
            auc_times = WebDriverWait(event, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="calendar_event_info"]'))) 
            auc_lots = WebDriverWait(event, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="calendar_event_vehicles"]'))) 
            if len(auc_name) == len(auc_times) == len(auc_lots):
                for auc, time_, lot in zip(auc_name, auc_times, auc_lots):
                    
                    details = {}
                    
                    details['Auction name'] = auc.text.strip()
                    details['Time']  = time_.text.strip().split("$")[-1]
                    details['Lots']  = lot.text.strip().split(" ")[0]
                    
                    result.append(details)
                    
            else:
                details['Auction name'] = 'na'
    except:
        print("no auc name found")
            

    df = pd.DataFrame.from_dict(result)
    df.to_csv("Auctions.csv", index=False)
    driver.quit()
    
path = "https://www.manheim.co.uk/catalogues-and-events"
scrape(path)
